In [ ]:
import pandas as pd
import geopandas as gpd
import db_dtypes
import matplotlib.pyplot as plt 
import folium
from shapely.geometry import Point, Polygon, LineString
from shapely import intersection
%matplotlib inline


In [ ]:
# create a simple polygon
p1 = Polygon([(0, 0), (1, 0), (1.25, 0.4), (2.3, 0.7), (1, 1), (1,1.3), (0, 1)])
#p2 = Polygon([(1, 0), (2, 0), (2, 1), (1, 1)])
p2 = Polygon([(1.5, 0), (3, 0), (3.3,1.5), (3, 2), (1.5, 2)])

attribs = {'id': ['A', 'B']}



In [ ]:
g = gpd.GeoDataFrame(attribs, geometry=[p1,p2])

In [ ]:
# visualize all polygons with their owner values
fig, ax = plt.subplots(figsize=(10, 10))
g.plot(ax=ax, edgecolor='black', facecolor='lightblue', alpha =0.7)

# Annotating each polygon with the 'owner' value at its centroid
for idx, row in g.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y, row['id'], horizontalalignment='center', fontsize=12, weight='bold')
    x, y = row.geometry.exterior.xy
    ax.plot(x, y, 'ro', markersize=3)
plt.show()

In [ ]:
# Perform a self-spatial join using the 'intersects' predicate
overlaps = gpd.sjoin(g, g, how="inner", predicate="overlaps")

# Filter out self-intersections
overlaps = overlaps[overlaps.index != overlaps.index_right]

overlaps

In [ ]:
data_overlaps=gpd.GeoDataFrame(columns=['id','geometry'],geometry='geometry')

for index, row in g.iterrows():
    row_geom = row.geometry
    row_id = row.id
    data_temp1 = g.loc[g.id!=row_id]
    print(f'data temp: {data_temp1.id}')
    # check if intersection occured
    overlaps=data_temp1[data_temp1.geometry.overlaps(row.geometry)]['id'].tolist()


    if len(overlaps)>0:
        temp_list=[]


        # compare the area with threshold

        
        for y in overlaps:
            overlap_geom = g.loc[g.id==y, 'geometry'].values[0]
            intersect_geom = intersection(row_geom, overlap_geom)
            #temp_area=gpd.overlay(,shapes.loc[shapes.id==row.id,],how='intersection')

            #data_overlaps=gpd.GeoDataFrame(pd.concat([temp_area,data_overlaps],ignore_index=True),crs=shapes.crs)
            #data_overlaps = pd.concat([data_overlaps,temp_area],ignore_index=True)

    break

In [ ]:
intersect_geom

In [ ]:
fixed_geom = g.symmetric_difference(intersect_geom)


In [ ]:
# visualize all polygons with their owner values
fig, ax = plt.subplots(figsize=(10, 10))
fixed_geom.plot(ax=ax, edgecolor='black', facecolor='lightblue', alpha =0.7)

plt.show()

In [ ]:
# Add extra vertices to help maintain overall shape of polygon
new_seg_geom = fixed_geom.segmentize(max_segment_length=0.2)

In [ ]:
new_seg_geom = new_seg_geom.buffer(50)
new_seg_geom = new_seg_geom.buffer(-50)

In [ ]:
# visualize all polygons with their owner values
fig, ax = plt.subplots(figsize=(10, 10))
new_seg_geom.plot(ax=ax, edgecolor='black', facecolor='lightblue', alpha =0.7)

plt.show()

In [ ]:
# Add extra vertices to help maintain overall shape of polygon
segment = g.segmentize(max_segment_length=0.3)

g2 = gpd.GeoDataFrame(attribs, geometry=segment)

g2=g2.set_crs(epsg=32610)
crs = g2.crs
unit = crs.axis_info[0].unit_name

#g2['geometry']= g2.geometry.buffer(0.5,cap_style='flat')
g2['geometry'] = g2.geometry.convex_hull
#g2['geometry'] = g2.geometry.concave_hull()

# visualize all polygons with their owner values
fig, ax = plt.subplots(figsize=(10, 10))
g2.plot(ax=ax, edgecolor='black', facecolor='lightblue')

# Annotating each polygon with the 'owner' value at its centroid
for idx, row in g2.iterrows():
    x, y = row.geometry.exterior.xy
    ax.plot(x, y, 'ro', markersize=3)



In [ ]:
simplify = g2.simplify(0.4)
print(simplify)
g3 = gpd.GeoDataFrame(attribs, geometry=simplify)

# visualize all polygons with their owner values
fig, ax = plt.subplots(figsize=(10, 10))
g3.plot(ax=ax, edgecolor='black', facecolor='lightblue')

# Annotating each polygon with the 'owner' value at its centroid
for idx, row in g3.iterrows():
    x, y = row.geometry.exterior.xy
    ax.plot(x, y, 'ro', markersize=3)


In [ ]:
g2['geometry']= g2.geometry.buffer(0.5,cap_style='flat')
# visualize all polygons with their owner values
fig, ax = plt.subplots(figsize=(10, 10))
g2.plot(ax=ax, edgecolor='black', facecolor='lightblue')

# Annotating each polygon with the 'owner' value at its centroid
for idx, row in g2.iterrows():
    x, y = row.geometry.exterior.xy
    ax.plot(x, y, 'ro', markersize=3)

In [ ]:
gdf = gpd.read_file("vexcel_selection.shp")


# Drop columns multiple column 
gdf = gdf.drop(['id_left', 'index_righ','id_right'], axis=1)
gdf



In [ ]:
# Get centroid locations
bounds = gdf.total_bounds
centroid_x = (bounds[0] + bounds[2]) / 2
centroid_y = (bounds[1] + bounds[3]) / 2
print(centroid_x, centroid_y)

In [ ]:
# Perform a self-spatial join using the 'intersects' predicate
overlaps = gpd.sjoin(gdf, gdf,how="inner", predicate="overlaps")

# Filter out self-intersections
overlaps = overlaps[overlaps.index != overlaps.index_right]

overlaps

In [ ]:
overlaps=overlaps.to_crs(epsg=32610)
simplify = overlaps.simplify(0.8)
print(simplify)
g3 = gpd.GeoDataFrame(attribs, geometry=simplify)

In [ ]:
# OpenStreetMap
map = folium.Map(location=[centroid_y, centroid_x], tiles="OpenStreetMap", zoom_start=25)
folium.GeoJson(overlaps).add_to(map)
#folium.GeoJson(g3).add_to(map)
map

In [ ]:

shapes = gpd.read_file("vexcel_selection.shp")
crs = shapes.crs

# OpenStreetMap
map = folium.Map(location=[centroid_y, centroid_x], tiles="OpenStreetMap", zoom_start=25)
folium.GeoJson(shapes).add_to(map)
map

In [ ]:
shapes.head()   


In [ ]:
shapes = gpd.read_file("vexcel_selection.shp")


In [ ]:

data_overlaps=gpd.GeoDataFrame(columns=['id','geometry'],geometry='geometry', crs=shapes.crs)

for index, row in shapes.iterrows():
    row_geom = row.geometry
    row_id = row.id
    data_temp1 = shapes.loc[shapes.id!=row_id]
    print(f'data temp: {data_temp1.id}')
    # check if intersection occured
    overlaps=data_temp1[data_temp1.geometry.overlaps(row.geometry)]['id'].tolist()


    if len(overlaps)>0:
        temp_list=[]


        # compare the area with threshold

        
        for y in overlaps:
            overlap_geom = shapes.loc[shapes.id==y, 'geometry'].values[0]
            intersect_geom = intersection(row_geom, overlap_geom)
            #temp_area=gpd.overlay(,shapes.loc[shapes.id==row.id,],how='intersection')

            #data_overlaps=gpd.GeoDataFrame(pd.concat([temp_area,data_overlaps],ignore_index=True),crs=shapes.crs)
            #data_overlaps = pd.concat([data_overlaps,temp_area],ignore_index=True)

    break
# data_overlaps.to_file('overlaps.shp')
# # get unique of list id
# data_overlaps['sorted']=data_overlaps.apply(lambda y: sorted([y['id_1'],y['id_2']]),axis=1)
# #data_overlaps['sorted']=data_overlaps.sorted.apply(lambda y: ''.join(y))
# #data_overlaps=data_overlaps.drop_duplicates('sorted')
# #data_overlaps=data_overlaps.reset_index()[['id_1','id_2','geometry']]

In [ ]:
intersect_geom

In [ ]:
# OpenStreetMap
map = folium.Map(location=[centroid_y, centroid_x], tiles="OpenStreetMap", zoom_start=25)
folium.GeoJson(intersect_geom).add_to(map)
#folium.GeoJson(g3).add_to(map)
map

In [ ]:
fixed_geom = shapes.symmetric_difference(intersect_geom)
# OpenStreetMap
map = folium.Map(location=[centroid_y, centroid_x], tiles="OpenStreetMap", zoom_start=25)
folium.GeoJson(fixed_geom).add_to(map)
map

#fixed_geom.to_file('fixed.shp')

In [ ]:
fixed_geom = fixed_geom.reset_index()
fixed_geom

In [ ]:
from shapely.ops import unary_union

In [ ]:
fixed_reproj = fixed_geom.to_crs(epsg=32610)
fixed_reproj.crs

In [ ]:
fixed_union = fixed_reproj.unary_union

#fixed_geom_buff = new.buffer(50).buffer(-50)

In [ ]:
[fixed_union]

In [ ]:
fixed_union_gdf = gpd.GeoDataFrame(geometry=[fixed_union], crs=32610)
fixed_union_gdf

In [ ]:
fixed_union_gdf['geometry'] = fixed_union_gdf['geometry'].buffer(50)
fixed_union_gdf['geometry'] = fixed_union_gdf['geometry'].buffer(-50)

In [ ]:
# OpenStreetMap
map = folium.Map(location=[centroid_y, centroid_x], tiles="OpenStreetMap", zoom_start=25)
folium.GeoJson(fixed_union_gdf).add_to(map)
map

In [ ]:
#new=fixed_geom.to_crs(epsg=32610)

for index, row in shapes.iterrows():
    geom = row['geometry']
    geom.buffer(-1, join_style="mitre").buffer(1, join_style="mitre")

In [ ]:
buffer2 = buff.buffer(-0.5, join_style="mitre").buffer(5, join_style="mitre")

In [ ]:
# OpenStreetMap
map = folium.Map(location=[centroid_y, centroid_x], tiles="OpenStreetMap", zoom_start=25)
folium.GeoJson(buffer2).add_to(map)
map